In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


*drop columns not necessary for analysis*

In [5]:
df.drop(columns=[
    'PassengerId','Name','Ticket','Cabin'
],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
X = df.drop(columns=['Survived'])
y = df['Survived']


In [7]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2,random_state=2
)

In [8]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [9]:
numerical_features = ['Age','Fare']
categorical_features = ['Sex','Embarked']

In [10]:
X_train.isnull().mean()

,0
Pclass,0.000000
Sex,0.000000
Age,0.207865
SibSp,0.000000
Parch,0.000000
Fare,0.000000
Embarked,0.002809


*missing values are present in Age and Embarked columns, so first step is to fill them*

In [11]:
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(drop='first',handle_unknown='ignore'))
])

In [12]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num',numerical_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ], remainder = 'passthrough'
)

In [13]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [15]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('classifier', LogisticRegression())])

In [16]:
param_grid = {
    'preprocessor__num__imputer__strategy' : ['mean','median'],
    'preprocessor__cat__imputer__strategy' : ['most_frequent','constant'],
    'classifier__C' : [0.1,1.0,10,100]
}

grid_search = GridSearchCV(clf,param_grid,cv=10)

In [17]:
grid_search.fit(X_train,y_train)
print(f"Best params: ")
print(grid_search.best_params_)

Best params: 
{'classifier__C': 0.1, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [18]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.815


In [20]:
type(grid_search.cv_results_)

dict

In [21]:
import pandas as pd
cv_results = pd.DataFrame(grid_search.cv_results_)

In [22]:
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.060105,0.036373,0.023731,0.013913,0.1,most_frequent,mean,"{'classifier__C': 0.1, 'preprocessor__cat__imp...",0.805556,0.888889,...,0.859155,0.788732,0.802817,0.802817,0.788732,0.774648,0.830986,0.814515,0.033474,1
1,0.060109,0.032441,0.022752,0.010892,0.1,most_frequent,median,"{'classifier__C': 0.1, 'preprocessor__cat__imp...",0.805556,0.888889,...,0.845070,0.788732,0.802817,0.802817,0.788732,0.774648,0.830986,0.813106,0.031822,2
2,0.046394,0.015644,0.018174,0.007430,0.1,constant,mean,"{'classifier__C': 0.1, 'preprocessor__cat__imp...",0.805556,0.888889,...,0.859155,0.774648,0.802817,0.802817,0.788732,0.774648,0.830986,0.813106,0.034799,2
3,0.023764,0.001926,0.010795,0.000553,0.1,constant,median,"{'classifier__C': 0.1, 'preprocessor__cat__imp...",0.805556,0.888889,...,0.830986,0.774648,0.802817,0.802817,0.788732,0.774648,0.830986,0.810290,0.031984,4
4,0.023442,0.000494,0.010934,0.000543,1.0,most_frequent,mean,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.791667,0.888889,...,0.802817,0.774648,0.816901,0.802817,0.774648,0.774648,0.830986,0.806084,0.032861,7
5,0.026830,0.006036,0.012711,0.005261,1.0,most_frequent,median,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.791667,0.888889,...,0.816901,0.774648,0.816901,0.802817,0.774648,0.760563,0.816901,0.804675,0.033861,8
6,0.023571,0.001474,0.010938,0.000796,1.0,constant,mean,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.791667,0.888889,...,0.802817,0.788732,0.816901,0.802817,0.774648,0.774648,0.830986,0.807492,0.031767,5
7,0.023986,0.000759,0.010802,0.000477,1.0,constant,median,"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.791667,0.888889,...,0.816901,0.788732,0.816901,0.802817,0.774648,0.760563,0.830986,0.807492,0.033588,6
8,0.025962,0.005212,0.011633,0.001914,10.0,most_frequent,mean,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.791667,0.888889,...,0.802817,0.774648,0.830986,0.788732,0.774648,0.760563,0.830986,0.804675,0.035575,8
9,0.024015,0.000686,0.010893,0.000359,10.0,most_frequent,median,"{'classifier__C': 10, 'preprocessor__cat__impu...",0.791667,0.888889,...,0.802817,0.788732,0.830986,0.788732,0.774648,0.746479,0.802817,0.801858,0.035611,13


In [24]:
cv_results = cv_results.sort_values("mean_test_score",
                                    ascending=False,
                                    )

In [26]:
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]

,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
0,0.1,most_frequent,mean,0.814515
1,0.1,most_frequent,median,0.813106
2,0.1,constant,mean,0.813106
3,0.1,constant,median,0.810290
6,1.0,constant,mean,0.807492
7,1.0,constant,median,0.807492
4,1.0,most_frequent,mean,0.806084
5,1.0,most_frequent,median,0.804675
8,10.0,most_frequent,mean,0.804675
10,10.0,constant,mean,0.803286


*by sorting acc to mean_test_score we get the highest grid_search best score on the top, with the best params*